In [8]:
# Task 1: Edge AI Prototype

import os

# Create the folders
os.makedirs("data/train/plastic", exist_ok=True)
os.makedirs("data/train/paper", exist_ok=True)
os.makedirs("data/val/plastic", exist_ok=True)
os.makedirs("data/val/paper", exist_ok=True)
os.makedirs("data/test/plastic", exist_ok=True)
os.makedirs("data/test/paper", exist_ok=True)

print("Folders created!")

Folders created!


In [3]:
from PIL import Image
import numpy as np

# Create 5 dummy images per class
for cls in ["plastic", "paper"]:
    for i in range(5):
        arr = np.random.randint(0, 255, (160,160,3), dtype=np.uint8)
        img = Image.fromarray(arr)
        img.save(f"data/train/{cls}/{cls}{i}.jpg")
        img.save(f"data/val/{cls}/{cls}{i}.jpg")
        img.save(f"data/test/{cls}/{cls}{i}.jpg")

print("Dummy images created!")

Dummy images created!


In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/train",
    image_size=(160,160),
    batch_size=16,
    label_mode="categorical"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/val",
    image_size=(160,160),
    batch_size=16,
    label_mode="categorical"
)

print("Classes:", train_ds.class_names)

Found 10 files belonging to 2 classes.
Found 10 files belonging to 2 classes.
Classes: ['paper', 'plastic']


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/train",
    image_size=(160,160),
    batch_size=16,
    label_mode="categorical"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/val",
    image_size=(160,160),
    batch_size=16,
    label_mode="categorical"
)

# ✅ Save class names BEFORE prefetch
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

# Now apply cache/prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(200).prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

# Build a simple CNN
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(160,160,3)),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=5)

# Evaluate
val_loss, val_acc = model.evaluate(val_ds, verbose=0)
print(f"Validation accuracy: {val_acc:.3f}")

Found 10 files belonging to 2 classes.
Found 10 files belonging to 2 classes.
Classes: ['paper', 'plastic']
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.7104 - val_accuracy: 0.5000 - val_loss: 1.4858
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 902ms/step - accuracy: 0.5000 - loss: 1.4858 - val_accuracy: 0.5000 - val_loss: 0.6993
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 583ms/step - accuracy: 0.5000 - loss: 0.6993 - val_accuracy: 0.5000 - val_loss: 0.7172
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 852ms/step - accuracy: 0.5000 - loss: 0.7172 - val_accuracy: 0.5000 - val_loss: 0.6946
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 929ms/step - accuracy: 0.5000 - loss: 0.6946 - val_accuracy: 0.5000 - val_loss: 0.6868
Validation accuracy: 0.500
